the aim of this notebook is to prepare the dataset.
We'd like to have multiple series of price.

We only need one price per day (CLOSE for example).

In [154]:
import pandas as pd
import numpy as np

In [155]:
def prepare_tab(tab, n):

    df = pd.DataFrame(eval( '[' + 'np.arange(n),' * (len(tab)-n+1) + ']' ))
    
    p = n
    while p > 0:
        i = n-p
        for k in range(len(tab)-n+1):
            print(n, p, n-p, k, i, tab[i])
            df[n-p]\
            [k] = \
            tab[i]
            i += 1
        p -= 1
    
    return df

print(prepare_tab(list(range(60)), 7))

7 7 0 0 0 0
7 7 0 1 1 1
7 7 0 2 2 2
7 7 0 3 3 3
7 7 0 4 4 4
7 7 0 5 5 5
7 7 0 6 6 6
7 7 0 7 7 7
7 7 0 8 8 8
7 7 0 9 9 9
7 7 0 10 10 10
7 7 0 11 11 11
7 7 0 12 12 12
7 7 0 13 13 13
7 7 0 14 14 14
7 7 0 15 15 15
7 7 0 16 16 16
7 7 0 17 17 17
7 7 0 18 18 18
7 7 0 19 19 19
7 7 0 20 20 20
7 7 0 21 21 21
7 7 0 22 22 22
7 7 0 23 23 23
7 7 0 24 24 24
7 7 0 25 25 25
7 7 0 26 26 26
7 7 0 27 27 27
7 7 0 28 28 28
7 7 0 29 29 29
7 7 0 30 30 30
7 7 0 31 31 31
7 7 0 32 32 32
7 7 0 33 33 33
7 7 0 34 34 34
7 7 0 35 35 35
7 7 0 36 36 36
7 7 0 37 37 37
7 7 0 38 38 38
7 7 0 39 39 39
7 7 0 40 40 40
7 7 0 41 41 41
7 7 0 42 42 42
7 7 0 43 43 43
7 7 0 44 44 44
7 7 0 45 45 45
7 7 0 46 46 46
7 7 0 47 47 47
7 7 0 48 48 48
7 7 0 49 49 49
7 7 0 50 50 50
7 7 0 51 51 51
7 7 0 52 52 52
7 7 0 53 53 53
7 6 1 0 1 1
7 6 1 1 2 2
7 6 1 2 3 3
7 6 1 3 4 4
7 6 1 4 5 5
7 6 1 5 6 6
7 6 1 6 7 7
7 6 1 7 8 8
7 6 1 8 9 9
7 6 1 9 10 10
7 6 1 10 11 11
7 6 1 11 12 12
7 6 1 12 13 13
7 6 1 13 14 14
7 6 1 14 15 15
7 6 1 15 16 16
7 6 1 16

In [156]:
import warnings
warnings.filterwarnings('ignore')

In [157]:
from sklearn import tree, svm, linear_model, neural_network, gaussian_process
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV, ParameterGrid, cross_validate
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd
import numpy as np
import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [158]:
TIME = 'time'
OPEN = 'open'
HIGH = 'high'
LOW = 'low'
CLOSE = 'close'
VWAP = 'vwap'
VOLUME = 'volume'
COUNT = 'count'

In [159]:
# Open the dataset
df_num = pd.read_csv("ohlc-btc-eur-day.csv")

In [ ]:
# OR open this dataset
CLOSE = "Close**"
df = pd.read_csv("bitcoin-price-all-2.csv", sep=',')
# Convert to numeric
def raw_data_to_numeric(raw_data):
    data = raw_data.replace(' ', '')
    data = data.replace(',', '.')
    if data == '0':
        return np.nan
    return float(data)
df_num = df[['Open*', 'High', 'Low', 'Close**', 'Volume', 'Market Cap']]
df_num = df_num.applymap(raw_data_to_numeric)
# Replace NaN by first known data (for `Volume`)
df_num = df_num.fillna(46862700)
# Read dates
dates = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%b %d, %Y'))
dates_num = dates.map(datetime.datetime.toordinal)
df_dated = df_num.copy()
df_dated.insert(0, 'Date', dates)
df_dated.sort_values(by=['Date'], ascending=True)
df_num = df_dated.copy()[['Open*', 'High', 'Low', 'Close**', 'Volume', 'Market Cap']]

In [160]:
# Select columns
df_num = df_num[[CLOSE]]


In [161]:
# Parameter
n = 15

In [162]:
# Prepare function

def prepare(tab, n):
    df_ = pd.DataFrame(eval( '[' + 'np.arange(n)*0.0,' * (len(tab)-n+1) + ']' ))
    
    p = n
    while p > 0:
        i = n-p
        for k in range(len(tab)-n+1):
            df_[n-p][k] = float(tab[i])
            i += 1
        p -= 1
    
    return df_



In [163]:
# Prepare the non-normalized data
print(df_num[CLOSE])
df_series = prepare(list(df_num[CLOSE]), n)

print(df_series)
print(df_series.describe())

0      8527.8
1      8364.9
2      8062.1
3      8163.3
4      8008.1
        ...  
715    5339.3
716    5585.0
717    5855.3
718    5753.4
719    6376.9
Name: close, Length: 720, dtype: float64
          0       1       2       3       4       5       6       7       8  \
0    8527.8  8364.9  8062.1  8163.3  8008.1  8422.1  8413.6  8282.5  8671.0   
1    8364.9  8062.1  8163.3  8008.1  8422.1  8413.6  8282.5  8671.0  8953.2   
2    8062.1  8163.3  8008.1  8422.1  8413.6  8282.5  8671.0  8953.2  8912.0   
3    8163.3  8008.1  8422.1  8413.6  8282.5  8671.0  8953.2  8912.0  8927.0   
4    8008.1  8422.1  8413.6  8282.5  8671.0  8953.2  8912.0  8927.0  8687.2   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
701  5304.8  5212.0  5338.9  5274.1  5290.8  5208.2  5797.6  5837.4  5830.0   
702  5212.0  5338.9  5274.1  5290.8  5208.2  5797.6  5837.4  5830.0  5765.1   
703  5338.9  5274.1  5290.8  5208.2  5797.6  5837.4  5830.0  5765.1  5552.7   
704  5274.1  52

In [164]:
# Scale data
scaler = preprocessing.StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df_num.values), columns=df_num.columns, index=df_num.index)

# Unscale data
def unscale(value, key=CLOSE):
    return value * np.std(df_num[key]) + np.mean(df_num[key])

In [165]:
# Prepare the normalized data
print(df_norm[CLOSE])
df_series_norm = prepare(list(df_norm[CLOSE]), n)

print(df_series_norm)
print(df_series_norm.describe())

0      1.041922
1      0.962462
2      0.814761
3      0.864125
4      0.788421
         ...   
715   -0.513376
716   -0.393528
717   -0.261680
718   -0.311385
719   -0.007252
Name: close, Length: 720, dtype: float64
            0         1         2         3         4         5         6  \
0    1.041922  0.962462  0.814761  0.864125  0.788421  0.990363  0.986217   
1    0.962462  0.814761  0.864125  0.788421  0.990363  0.986217  0.922268   
2    0.814761  0.864125  0.788421  0.990363  0.986217  0.922268  1.111772   
3    0.864125  0.788421  0.990363  0.986217  0.922268  1.111772  1.249425   
4    0.788421  0.990363  0.986217  0.922268  1.111772  1.249425  1.229328   
..        ...       ...       ...       ...       ...       ...       ...   
701 -0.530205 -0.575471 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825   
702 -0.575471 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825 -0.270411   
703 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825 -0.270411 -0.274021   
704 -0.545180

In [166]:
# Train model

X = df_series_norm.iloc[:,:n-1]
y = df_series_norm.iloc[:,n-1]

print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel())
model = neural_network.MLPRegressor(hidden_layer_sizes=(100,),
                                    activation='relu',
                                    solver='lbfgs',
                                    alpha=1
                                   )

model.fit(X_train, y_train)

            0         1         2         3         4         5         6  \
0    1.041922  0.962462  0.814761  0.864125  0.788421  0.990363  0.986217   
1    0.962462  0.814761  0.864125  0.788421  0.990363  0.986217  0.922268   
2    0.814761  0.864125  0.788421  0.990363  0.986217  0.922268  1.111772   
3    0.864125  0.788421  0.990363  0.986217  0.922268  1.111772  1.249425   
4    0.788421  0.990363  0.986217  0.922268  1.111772  1.249425  1.229328   
..        ...       ...       ...       ...       ...       ...       ...   
701 -0.530205 -0.575471 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825   
702 -0.575471 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825 -0.270411   
703 -0.513571 -0.545180 -0.537034 -0.577325 -0.289825 -0.270411 -0.274021   
704 -0.545180 -0.537034 -0.577325 -0.289825 -0.270411 -0.274021 -0.305678   
705 -0.537034 -0.577325 -0.289825 -0.270411 -0.274021 -0.305678 -0.409283   

            7         8         9        10        11        12        13  

MLPRegressor(alpha=1, solver='lbfgs')

In [167]:
def absolute_error_score(y_true, y_pred):
    diff = np.abs(unscale(y_true) - unscale(y_pred))
    return -np.average(diff, axis=0)

def absolute_error(estimator, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    return absolute_error_score(y_test, y_pred)

def percentage_error_score(y_true, y_pred):
    percentage = (np.abs(y_pred - y_true)) / np.abs(y_true) * 100
    return -np.average(percentage, axis=0)

def percentage_error(estimator, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    return percentage_error_score(y_test, y_pred)

def aon(x_close, y_pred, y_true):
    # all-or-none principle
    return not ((y_pred <= x_close) != (y_true < x_close))
        
def aon_scorer(estimator, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test).reshape(X_test.shape[0], 1)
    X_estimator = X_test.copy()
    X_estimator.insert(n-1, 'y_test', y_test)
    X_estimator.insert(n, 'y_pred', y_pred)
    aon_series = X_estimator.apply(lambda val: aon(val[n-2], val['y_pred'], val['y_test']), axis=1)
    return np.mean(aon_series)





In [168]:
model = neural_network.MLPRegressor(hidden_layer_sizes=(100,),
                                    activation='relu',
                                    solver='lbfgs',
                                    alpha=1
                                   )

scores = cross_validate(model, X, y,
                        scoring={
                            'percentage_error': percentage_error,
                            'absolute_error': absolute_error,
                            'aon_scorer': aon_scorer
                        })

print(np.mean(scores['test_absolute_error']), np.std(scores['test_absolute_error']))
print(np.mean(scores['test_percentage_error']), np.std(scores['test_percentage_error']))
print(np.mean(scores['test_aon_scorer']), np.std(scores['test_aon_scorer']))
scores

-199.19342228995237 41.526153734084694
-43.61534397708244 38.65184388183109
0.47777777777777775 0.07934920476158723


{'fit_time': array([1.00143647, 0.70846081, 0.66917539, 0.59097886, 0.53923655]),
 'score_time': array([1.05541229, 0.60668349, 0.99685073, 0.63606215, 0.36306119]),
 'test_percentage_error': array([-91.32848138, -89.54203842, -13.25914335,  -3.0261118 ,
        -20.92094493]),
 'test_absolute_error': array([-200.74962567, -172.00834817, -269.33009999, -145.54659607,
        -208.33244155]),
 'test_aon_scorer': array([0.61111111, 0.52777778, 0.41666667, 0.41666667, 0.41666667])}

In [169]:
model = neural_network.MLPRegressor(hidden_layer_sizes=(10,10,10),
                                    activation='identity',
                                    solver='lbfgs',
                                    alpha=0.01
                                   )

scores = cross_validate(model, X, y,
                        scoring={
                            'percentage_error': percentage_error,
                            'absolute_error': absolute_error,
                            'aon_scorer': aon_scorer
                        })

print(np.mean(scores['test_absolute_error']), np.std(scores['test_absolute_error']))
print(np.mean(scores['test_percentage_error']), np.std(scores['test_percentage_error']))
print(np.mean(scores['test_aon_scorer']), np.std(scores['test_aon_scorer']))
scores

-198.09290110353498 61.73881179831931
-70.3341976879078 100.56899020868951
0.5 0.07027283689263066


{'fit_time': array([0.10888863, 0.06552911, 0.05637002, 0.04784989, 0.09430027]),
 'score_time': array([0.24875998, 0.26210237, 0.28466821, 0.32289028, 0.22849584]),
 'test_percentage_error': array([ -44.37533827,  -15.97804273,  -15.20099991,   -6.28316118,
        -269.83344635]),
 'test_absolute_error': array([-287.56229057, -187.58375022, -245.08827672, -155.14278487,
        -115.08740313]),
 'test_aon_scorer': array([0.52777778, 0.55555556, 0.52777778, 0.36111111, 0.52777778])}

In [103]:
############################################# Search the best parameters
model = neural_network.MLPRegressor()

gs = GridSearchCV(
    model,
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'alpha': 10.0**np.arange(-2,2),
        'solver': ['lbfgs', 'sgd', 'adam'],
        'hidden_layer_sizes': [(100,), (10, 10, 10), (125, 125)]
    },
    scoring=absolute_error
)

gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.score(X_test, y_test))

{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (100,), 'solver': 'lbfgs'}
-191.23155356663693


In [104]:
############################################# Search the best parameters
model = neural_network.MLPRegressor()

gs = GridSearchCV(
    model,
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'alpha': 10.0**np.arange(-2,2),
        'solver': ['lbfgs', 'sgd', 'adam'],
        'hidden_layer_sizes': [(100,), (10, 10, 10), (125, 125)]
    },
    scoring=percentage_error
)

gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.score(X_test, y_test))

{'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (100,), 'solver': 'lbfgs'}
-30.876364804634445


In [105]:
############################################# Search the best parameters
model = neural_network.MLPRegressor()

gs = GridSearchCV(
    model,
    {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'alpha': 10.0**np.arange(-2,2),
        'solver': ['lbfgs', 'sgd', 'adam'],
        'hidden_layer_sizes': [(100,), (10, 10, 10), (125, 125)]
    },
    scoring=aon_scorer
)

gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.score(X_test, y_test))

{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (10, 10, 10), 'solver': 'lbfgs'}
0.4888888888888889
